In [1]:
# 모듈 import

from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.utils import get_device

# wandb 코드 추가
import wandb

wandb.init(project='KO_project', name='yolo base')

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


In [4]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
# cfg = Config.fromfile('./configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py')
cfg = Config.fromfile('./configs/yolo/yolov3_d53_320_273e_coco.py')
# wandb 코드 추가
cfg.log_config.hooks = [
    dict(type='TextLoggerHook'),
    dict(type='MMDetWandbHook',
         init_kwargs={'project' : 'KO_project'},
         interval = 10,
         log_checkpoint=True,
         log_checkpoint_metadata=True,
         num_eval_images=100)
]

root='../../../dataset/'

# dataset config 수정
cfg.data.train.classes = classes
cfg.data.train.img_prefix = root
cfg.data.train.ann_file = root + 'train.json' # train json 정보
cfg.data.train.pipeline[4]['img_scale'] = (512,512) # Resize

cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json' # test json 정보
cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize

cfg.data.samples_per_gpu = 4

cfg.seed = 2022
cfg.gpu_ids = [0]
# cfg.work_dir = './work_dirs/faster_rcnn_r50_fpn_1x_trash_'
cfg.work_dir = './work_dirs/fyolo/yolov3_d53_320_273e_trash_'


# cfg.model.roi_head.bbox_head.num_classes = 10
cfg.model.bbox_head.num_classes = 10

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.checkpoint_config = dict(max_keep_ckpts=3, interval=1)
cfg.device = get_device()

In [5]:
# build_dataset
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=0.09s)
creating index...
index created!


In [6]:
# dataset 확인
datasets[0]


CocoDataset Train dataset with number of images 4883, and instance counts: 
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| category          | count | category      | count | category        | count | category    | count | category     | count |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| 0 [General trash] | 3965  | 1 [Paper]     | 6352  | 2 [Paper pack]  | 897   | 3 [Metal]   | 936   | 4 [Glass]    | 982   |
| 5 [Plastic]       | 2943  | 6 [Styrofoam] | 1263  | 7 [Plastic bag] | 5178  | 8 [Battery] | 159   | 9 [Clothing] | 468   |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+

In [7]:
# 모델 build 및 pretrained network 불러오기
model = build_detector(cfg.model)
model.init_weights()

2024-01-08 16:52:54,852 - mmcv - INFO - initialize Darknet with init_cfg {'type': 'Pretrained', 'checkpoint': 'open-mmlab://darknet53'}
2024-01-08 16:52:54,854 - mmcv - INFO - load model from: open-mmlab://darknet53
2024-01-08 16:52:54,855 - mmcv - INFO - load checkpoint from openmmlab path: open-mmlab://darknet53
2024-01-08 16:52:55,214 - mmcv - INFO - 
backbone.conv1.conv.weight - torch.Size([32, 3, 3, 3]): 
PretrainedInit: load from open-mmlab://darknet53 
 
2024-01-08 16:52:55,215 - mmcv - INFO - 
backbone.conv1.bn.weight - torch.Size([32]): 
PretrainedInit: load from open-mmlab://darknet53 
 
2024-01-08 16:52:55,216 - mmcv - INFO - 
backbone.conv1.bn.bias - torch.Size([32]): 
PretrainedInit: load from open-mmlab://darknet53 
 
2024-01-08 16:52:55,217 - mmcv - INFO - 
backbone.conv_res_block1.conv.conv.weight - torch.Size([64, 32, 3, 3]): 
PretrainedInit: load from open-mmlab://darknet53 
 
2024-01-08 16:52:55,217 - mmcv - INFO - 
backbone.conv_res_block1.conv.bn.weight - torch.Siz

In [8]:
# 모델 학습
train_detector(model, datasets[0], cfg, distributed=False, validate=False)

2024-01-08 16:52:59,796 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
2024-01-08 16:52:59,801 - mmdet - INFO - Start running, host: root@instance-5190, work_dir: /data/ephemeral/home/level2-objectdetection-cv-10/develop/Ko/mmdetection/work_dirs/fyolo/yolov3_d53_320_273e_trash_
2024-01-08 16:52:59,802 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) MMDetWandbHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) MMDetWandbHook                     
 -------------------- 
before_train_iter:
(VERY_HIGH   ) StepLrUpdaterHook                

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


Problem at: /opt/conda/lib/python3.10/site-packages/mmcv/runner/hooks/logger/wandb.py 83 before_run


KeyboardInterrupt: 